### Import required libraries

In [32]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.svm import SVC

### Import the dataframes

In [7]:
modeling = pd.read_pickle('C:/Users/andre/Documents/Strive_repository/Predicting_nfl_games/data analysis/modeling.pkl')
forecast = pd.read_pickle('C:/Users/andre/Documents/Strive_repository/Predicting_nfl_games/data analysis/forecast.pkl')

In [10]:
# W or L into differential

modeling['Differential'] = modeling['Differential'].apply(lambda x: 1 if x > 0 else 0)
modeling.head(3)

,Differential,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Chiefs Texans 2020,1,90.2,84.5,86.1,65.7,87.6,71.2,71.9,50.4,75.4,...,59.8,75.1,64.5,68.6,76.7,59.4,87.0,67.3,59.1,53.4
Seahawks Falcons 2020,1,92.0,81.9,79.6,77.9,68.4,74.1,87.6,76.4,73.4,...,90.1,79.7,56.8,76.6,72.0,53.0,72.0,72.1,87.7,47.0
Bills Jets 2020,1,64.1,78.3,60.2,69.3,65.1,73.1,74.3,65.6,77.7,...,81.3,74.4,79.0,73.9,87.9,60.6,73.1,72.0,56.1,65.6


### Split the data

In [21]:
# Split the data

X = modeling.drop('Differential', axis= 1)
Y = modeling['Differential']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state= 0)

,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,RT FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Cowboys Cardinals 2021,85.2,75.9,61.4,71.3,76.5,76.6,62.7,89.8,72.0,86.4,...,85.4,88.3,74.5,59.9,75.3,69.0,86.0,73.6,64.8,56.7
Steelers Texans 2020,78.2,69.3,71.1,73.6,91.5,74.0,80.5,76.2,79.8,75.9,...,59.8,75.1,64.5,68.6,76.7,59.4,87.0,67.3,59.1,53.4
Bengals Chargers 2021,75.1,83.1,61.4,76.7,69.6,70.1,67.0,54.7,90.1,71.4,...,70.6,83.6,63.1,66.6,82.3,61.6,90.2,86.4,65.9,66.6
Cowboys Vikings 2020,80.1,84.1,61.4,82.9,72.0,76.6,72.2,70.2,74.2,86.0,...,69.1,66.2,70.1,90.1,91.1,70.8,89.0,57.7,69.5,33.1
49ers Giants 2020,77.5,77.5,94.4,82.8,89.6,75.6,90.3,67.7,84.9,70.7,...,78.1,62.4,61.7,72.8,66.6,64.8,60.8,56.7,58.4,76.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Commanders Eagles 2021,75.1,78.5,65.2,78.6,78.4,74.9,71.4,64.8,84.1,74.9,...,74.1,70.3,62.9,60.2,70.8,71.9,81.5,69.6,62.4,92.8
49ers Texans 2021,67.5,80.1,84.9,79.3,61.3,91.9,77.6,88.6,73.5,79.7,...,52.1,75.4,71.5,67.7,65.5,62.1,70.9,62.0,52.1,70.1
Panthers Cowboys 2021,58.4,79.1,61.7,86.5,68.0,61.6,70.4,83.3,59.0,81.6,...,76.5,76.6,62.7,89.8,72.0,86.4,88.6,53.5,70.8,91.3


### Preprocess

In [24]:
# Preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)

train_scaled = scaler.transform(X_train)
test_scaled = scaler.transform(X_test)
X_scaled1 = scaler.transform(X)

scaler2 = preprocessing.StandardScaler().fit(X)
X_scaled2 = scaler2.transform(X)
forecast_scaled = scaler2.transform(forecast)


### Prediction

In [31]:
AB = BaggingClassifier(base_estimator= SVC(tol= .1, C= .9), n_jobs= -1, n_estimators= 50).fit(train_scaled, Y_train)

print(np.sum(np.array(AB.predict(train_scaled)) == np.array(Y_train)) / len(Y_train))
print(np.sum(np.array(AB.predict(test_scaled)) == np.array(Y_test)) / len(X_test))

0.803030303030303
0.625


In [33]:
# Cross Validation

cvscore = cross_val_score(AB, X_scaled1, Y, cv= 10)
np.average(cvscore)

0.6563072776280323

### Forecasting 2022 season

In [34]:
AB = BaggingClassifier(base_estimator = SVC(tol = .1, C = .9), n_jobs = -1, n_estimators = 50).fit(X_scaled2,Y)
forecast["result"] = AB.predict(forecast_scaled)

In [35]:
forecast["result"] = forecast["result"].apply(lambda x: "W" if x > 0 else "L")
forecast["result"].to_frame().to_csv("2022predictions.csv")

In [36]:
teams = ["Buccaneers", "Chiefs", "Bills",  "Packers", "Ravens", "Rams", "Seahawks", "Browns", "49ers", "Steelers", "Titans", "Chargers", "Saints", "Patriots", "Dolphins", "Colts", "Vikings", "Cardinals", "Bears", "Cowboys", "Raiders", "Commanders", "Giants", "Broncos", "Falcons", "Eagles", "Panthers", "Jets", "Jaguars", "Bengals", "Lions", "Texans"]
rankdict = {}
for i in range(1, 33):
        rankdict[teams[i - 1]] = i
season2022 = X.tail(272)
results2022 = np.array(Y.tail(272))
season2022 = season2022.reset_index()
predictresults = np.array(season2022["gamename"].apply(lambda x: rankdict[x.split(" ")[0]] < rankdict[x.split(" ")[1]]))
sum(predictresults.astype(int) == results2022)/272

0.6286764705882353